In [1]:
from transformers import pipeline
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("FacebookSentimentAnalysis").getOrCreate()

# Initialize the transformer pipeline and broadcast it
model = pipeline("text-classification", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", top_k=None)
model_broadcast = spark.sparkContext.broadcast(model)


StatementMeta(, a7fe9b95-8851-4b40-8012-ddc688416367, 3, Finished, Available)

2024-06-20 10:50:26.759709: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 10:50:32.770512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

StatementMeta(, a7fe9b95-8851-4b40-8012-ddc688416367, 7, Finished, Available)

In [2]:
fact_facebook = spark.sql("SELECT * FROM P2.facebook_data")
display(fact_facebook)

StatementMeta(, a7fe9b95-8851-4b40-8012-ddc688416367, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, b00459a0-67b3-40f8-8eb1-2df15c4f9c0b)

In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

def get_sentiment(comment):
    model = model_broadcast.value
    if comment is None:
        comment = ''
    result = model(comment)[0]
    # Remove neutral label
    result = [r for r in result if r["label"] != "neutral"]
    # Get the label with the maximum score
    if result:
        sentiment = max(result, key=lambda x: x['score'])['label']
        return sentiment == "positive"
    return False  # Default to False if no non-neutral sentiment found

# Register the UDF
sentiment_udf = udf(get_sentiment, BooleanType())


StatementMeta(, a7fe9b95-8851-4b40-8012-ddc688416367, 5, Finished, Available)

In [4]:
# Apply the UDF to add a new column "sentiment" to the DataFrame
fact_facebook_with_sentiment = fact_facebook.withColumn("sentiment", sentiment_udf(fact_facebook.comment))
display(fact_facebook_with_sentiment)


StatementMeta(, a7fe9b95-8851-4b40-8012-ddc688416367, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 66f23180-b17b-4938-bc13-b8e54a0198ad)

In [6]:
# Define the delta table name
delta_table_name = "fact_facebook_with_sentiment"

# Write the DataFrame with sentiment to the Delta table
fact_facebook_with_sentiment.write.format("delta").mode("overwrite").option("overwriteSchema", True).saveAsTable(delta_table_name)
print(f"DataFrame has been written to Delta table: {delta_table_name}")


StatementMeta(, eadadbd7-93dc-4e0b-9e17-f6dcf2e62032, 13, Finished, Available)